In [25]:
'''
1. Numbers
2. Apostrophe
3. All punctuations
4. Weird symbols
5. Stop words
6. lemmatization
'''
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from sklearn.feature_extraction import stop_words
from nltk.stem.wordnet import WordNetLemmatizer
sets=[stop_words.ENGLISH_STOP_WORDS]
sklearnStopWords = [list(x) for x in sets][0]
token=ToktokTokenizer()
lemma=WordNetLemmatizer()
stopWordList=stopwords.words('english')
stopWords = stopWordList + sklearnStopWords
stopWords = list(dict.fromkeys(stopWords))

file_name = 'TED_Transcripts_short.csv'
df = pd.read_csv('C:/Users/JSaw/Desktop/50.038/TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv')
df = df.dropna(subset=['transcript'])
df = df.reset_index(drop=True)

def stopWordsRemove(text):
    wordList=[x.lower().strip() for x in token.tokenize(text)]
    removedList=[x + ' ' for x in wordList if not x in stopWords]
    text=''.join(removedList)
    return text


def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w,'v')
        listLemma.append(x)
    return text


# There is a mispelt word that needs to be replaced
df['transcript'] = df['transcript'].str.replace('\r',' ')
df['transcript'] = df['transcript'].str.replace("\'s"," is")
df['transcript'] = df['transcript'].str.replace("\'m"," am")
df['transcript'] = df['transcript'].str.replace("\'ll"," will")
df['transcript'] = df['transcript'].str.replace("Can\'t","cannot")
df['transcript'] = df['transcript'].str.replace("Sha\'t","shall not")
df['transcript'] = df['transcript'].str.replace("Won\'t","would not")
df['transcript'] = df['transcript'].str.replace("n\'t"," not")
df['transcript'] = df['transcript'].str.replace("\'ve"," have")
df['transcript'] = df['transcript'].str.replace("\'re"," are")
df['transcript'] = df['transcript'].str.replace("\'d"," would")
df['transcript'] = df['transcript'].str.replace(r"\(([^)]+)\)","")
# # Deal with Mr. and Dr.
# df['transcript'] = df['transcript'].str.replace("mr.","mr")
# df['transcript'] = df['transcript'].str.replace("Mr.","mr")
# df['transcript'] = df['transcript'].str.replace("mrs.","mrs")
# df['transcript'] = df['transcript'].str.replace("Mrs.","mrs")
# df['transcript'] = df['transcript'].str.replace("Dr.","dr")
# df['transcript'] = df['transcript'].str.replace("dr.","dr")
# df['transcript'] = df['transcript'].str.replace("childrn","children")
# df['transcript'] = df['transcript'].str.replace(r'\d+','')
# df['transcript'] = df['transcript'].str.replace(r'<.*?>','')
for i in string.punctuation:
    if i == "'":
        df['transcript'] = df['transcript'].str.replace(i,'')
    else:
        df['transcript'] = df['transcript'].str.replace(i,' ')
df['transcript'] = df['transcript'].map(lambda com : stopWordsRemove(com))
df['transcript'] = df['transcript'].map(lambda com : lemitizeWords(com))
df['transcript'] = df['transcript'].str.replace('\s+',' ')


In [26]:
df['tags'][0]

'cars,alternative energy,culture,politics,science,climate change,environment,sustainability,global issues,technology'

In [27]:
tags = df['tags'].str.replace(', ', ',').str.lower().str.strip()
split_tags = tags.str.split(',')
tag_counts_per_talk = split_tags.apply(len)

joined_tags = tags.str.cat(sep=',').split(',')
all_tags_w_dup = pd.Series(joined_tags)

tag_counts = all_tags_w_dup.value_counts()

# df['tags'] = split_tags

tag_cutoff = int(0.01*len(df.index))
print(tag_cutoff)
squashed_tags = pd.DataFrame(tag_counts)
# squashed_tags.columns = ['tag','count']
squashed_tags = squashed_tags[(squashed_tags[0]>tag_cutoff)]
squash_list = list(squashed_tags.index.values)
print(len(squash_list))

23
179


In [30]:
for i in range(len(df.index)):
    original_tags = df['tags'][i]
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    df['tags'][i] = final_string

C:\Users\JSaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [35]:
df[df['tags']=='']

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,...,harm_vice,fairness_virtue,fairness_vice,ingroup_virtue,ingroup_vice,authority_virtue,authority_vice,purity_virtue,purity_vice,morality_general


In [32]:
df.to_pickle("./squashed_processed_data.pkl")

In [36]:
df['tags']

0       alternative energy,culture,politics,science,cl...
1       industrial design,alternative energy,invention...
2       poverty,economics,investment,culture,politics,...
3       industrial design,invention,engineering,entrep...
4       industrial design,transportation,invention,des...
5       biotech,invention,oceans,genetics,dna,biology,...
6       computers,software,interface design,music,medi...
7       death,culture,architecture,cities,urban planni...
8       robots,industrial design,transportation,invent...
9       disease,health,health care,culture,food,scienc...
10      africa,culture,science,environment,animals,nat...
11               cancer,disease,health,science,technology
12      invention,culture,architecture,design,creativi...
13      invention,software,music,entertainment,perform...
14      love,gender,relationships,cognitive science,ps...
15      dna,evolution,biology,fish,science,environment...
16      philosophy,evolution,culture,choice,history,sc...
17      econom